In [ ]:
import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

# Make your layout in Qiskit Metal

In [ ]:
from qiskit_metal import Dict, designs, MetalGUI
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.couplers.quad_coupler import QuadCoupler

In [ ]:
design = designs.MultiPlanar({}, overwrite_enabled=True, layer_stack_filename="flip_chip_layer_stack.txt")
gui = MetalGUI(design)

In [ ]:
# Display the layer stack loaded from the file
design.ls.ls_df

In [ ]:
# Delete the pre-existing chips ('main' in this case)
design.chips.pop('main')

# Add the two chips manually
design.chips['c_chip'] = {'size': {'center_x': '0.0mm',
                                   'center_y': '0.0mm',
                                   'size_x': '9mm',
                                   'size_y': '7mm'}}

design.chips['q_chip'] = {'size': {'center_x': '0.0mm',
                                   'center_y': '0.0mm',
                                   'size_x': '9mm',
                                   'size_y': '7mm'}}

design.chips

In [ ]:
design.delete_all_components()

In [ ]:
conn_pads = dict(
    connection_pads = dict(
        coup1 = dict(loc_W=-1, loc_H=1),
        coup2 = dict(loc_W=1, loc_H=1)
    )
)

q1 = TransmonPocket6(design, "Q1", options=dict(**conn_pads, chip="q_chip", layer=2))

gui.rebuild()
gui.autoscale()

In [ ]:
qc1 = QuadCoupler(design, "qc1", options=dict(pos_x="0mm",
                                              pos_y="-0.08mm",
                                              pad_width="120um",
                                              pad_height="30um",
                                              cpw_stub_height="250um",
                                              chip="c_chip",
                                              layer=1))

gui.rebuild()
gui.autoscale()

# Simulation with Gmsh and ElmerFEM

In [ ]:
from qiskit_metal.renderers.renderer_gmsh.gmsh_renderer import QGmshRenderer

gmsh = QGmshRenderer(design, layer_types=dict(metal=[1, 2], dielectric=[3, 4]))
gmsh.render_design(mesh_geoms=False)
gmsh.launch_gui()
gmsh.close()